In [1]:
import pandas as pd
import numpy as np 
import os
import altair as alt
import tabula 
import html5lib

In [20]:
pd.options.mode.chained_assignment = None

In [268]:
dfs = []
raw = []
files = [f for f in os.listdir("../data") if f.endswith('html')]

for file in files:
    target = pd.read_html(f"../data/{file}")[-1]
    raw.append(target)
    
    disabled = target.iloc[1:8, :]
    disabled['disability_status'] = 'disabled'
    vals = disabled[disabled['Disability Status/ Insurance Type'] == 'Uninsured'].values[0]
    pct_insured = 100 - float(vals[1])
    disabled_insured_row = ['Insured', pct_insured]
    disabled_insured_row.extend(vals[2:])
    disabled_insured_row = pd.DataFrame([disabled_insured_row], columns = disabled.columns)
    disabled = pd.concat([disabled, disabled_insured_row])
    
    not_disabled = target.iloc[9:, :]
    not_disabled['disability_status'] = 'not_disabled'
    vals = not_disabled[not_disabled['Disability Status/ Insurance Type'] == 'Uninsured'].values[0]
    pct_insured = 100 - float(vals[1])
    not_disabled_insured_row = ['Insured', pct_insured]
    not_disabled_insured_row.extend(vals[2:])
    not_disabled_insured_row = pd.DataFrame([not_disabled_insured_row], columns = not_disabled.columns)
    not_disabled = pd.concat([not_disabled, not_disabled_insured_row])
    
    
    target = pd.concat([not_disabled, disabled])
    target['year'] = int(file.split("_")[-1][:4])
    dfs.append(target)

    
df = pd.concat(dfs).reset_index(drop = True)
raw_df = pd.concat(raw).reset_index(drop = True)

In [269]:
targets = ['Employer/Union', 'Purchased', 'Medicare', 'Medicaid', 'Insured', 'Any Disability', 'Uninsured']

In [270]:
#df = df[df['Disability Status/ Insurance Type'].isin(targets)]
df = df.rename(columns = {'Disability Status/ Insurance Type': 'type'})

In [271]:
insured = df[(df.type == 'Insured') & (df.disability_status == 'disabled')]
uninsured = df[(df.type == 'Uninsured') & (df.disability_status == 'disabled')]
employer = df[(df.type == 'Employer/Union') & (df.disability_status == 'disabled')]
purchased = df[(df.type == 'Purchased') & (df.disability_status == 'disabled')]
medicare = df[(df.type == 'Medicare') & (df.disability_status == 'disabled')]
medicaid = df[(df.type == 'Medicaid') & (df.disability_status == 'disabled')]

In [284]:
disabled = df[(df.disability_status == 'disabled') & (df.type != 'Insured') & (df.type != 'Uninsured')]

all_uninsured = df[df.type == 'Uninsured']

In [325]:
highlight = alt.selection_single(on = 'mouseover', fields = ['type'], nearest = True)
color_condition = alt.condition(highlight, alt.Color('type:N'), alt.value('gray'))

aca_df = pd.DataFrame({'year': [2010]})
aca_effect_df = pd.DataFrame({'year': [2014]})

uninsured_rates = alt.Chart(all_uninsured).mark_line().encode(
    x = alt.X('year:O'), 
    y = alt.Y('Percent:Q'), 
    color = alt.Color('disability_status:O')
)

aca_passage = alt.Chart(aca_df).mark_rule(
    color = 'lightblue', 
    size = 2.5
).encode(
    x = alt.X('year:O')
)

aca_passage_text = alt.Chart(aca_df).mark_text(
    text = "The Affordable Care Act \n(aka 'Obamacare') passes",
    fontSize = 10, 
    dy = 135
)

aca_effect = alt.Chart(aca_effect_df).mark_rule(
    color = 'lightblue', 
    size = 2.5
).encode(
    x = alt.X('year:O')
)

aca_effect_text = alt.Chart(aca_effect_df).mark_text(
    text = 'Most ACA Provisions Go Into Effect', 
    fontSize = 10, 
    dy = 135
)

insured_lines = alt.Chart(disabled).mark_line().encode(
    x = alt.X('year:O'),
    y = alt.Y('Percent:Q'),
    color = alt.Color('type:N')
)

insured_points = alt.Chart(disabled).add_selection(
    highlight
).mark_circle().encode(
    x = alt.X('year:O'),
    y = alt.Y('Percent:Q'),
    opacity = alt.value(0)
)

insured_lines = insured_lines.encode(
    color = color_condition
)

alt.layer(uninsured_rates, aca_passage, aca_effect, aca_passage_text, aca_effect_text
).properties(width = 350)

#| insured_points + insured_lines

alt.LayerChart(...)

In [ ]:
alt.Chart(medicare).add_selection(selection_2).mark_point().encode(
    x = alt.X('year:O'), 
    y = alt.Y('Percent:Q'), 
    color = color_selection
)